## Map generator Versie 1
In dit bestand leg ik uit hoe ik maps genereer. Als eerste moet ik een algoritme kiezen om paden te genereren op een 2d map space, deze paden moeten een bepaalde dikte hebben.

- Bronnen:
    - https://theory.stanford.edu/~amitp/GameProgramming/AStarComparison.html
    - https://medium.com/@yvanscher/playing-with-perlin-noise-generating-realistic-archipelagos-b59f004d8401
    - https://www.geeksforgeeks.org/find-whether-path-two-cells-matrix/
    - https://www.redblobgames.com/pathfinding/a-star/introduction.html

Versie 1 is de versie waar ik een map kan genereren, deze map is niet helemaal wat ik hier van verwacht, doordat ik met perlin noise mijn funcite maak en erna kijk of er een pad van links naar rechts is worden maps met veel wit in het midden bevoordeeld, in een 2e versie wil ik dit gaan verbeteren.

Als eerste ben ik begonnen met een map genereren doormiddel van een noise functie, hiermee krijg ik een simpele map die wij als groepje kunnen gebruiken om alvast te proberen om de intergratie werkend te maken. De bedoeling is dat de map in een 2d array komt, deze array heeft verschillende nummers die geassiocieerd worden met verschillende tiles.

In [2]:
import noise
import numpy as np
from PIL import Image
import random, sys, json

#### Helper functies
Hieronder staan wat algemene functies die ik meerdere keren nodig heb gehad die niet per se in een bepaald stukje passen. Dit stukje moet gerund worden omdat de meeste stukjes code wel een van deze functies gebruiken.

In [3]:
# zet alle values van de map die hoger zijn dan 0 naar een bepaald value.
def set_map_numbers(map, value):
    for i in range(len(map)):
        for n in range(len(map[0])):
            if map[i][n] > 0:
                map[i][n] = value
            else:
                map[i][n] = 0
    return map


Voor de eerste versie van de map gebruik ik nu een noise generator, perlin noise, dit gebruik ik om makkelijk te kunnen testen voor de implentmentatie met Unity, de API en de database. De noise generator maakt een value tussen -1 en 1 voor dit project ben ik op zoek naar de values 0 en 1, dus loop ik over de array om te checken in welke catogory dit value past. Ook wil ik kijken of de map valid is, dit wil ik doen door te kijken of er een pad is van links midden naar rechts midden door alleen maar nullen. <br> BFS (breadth first )

In [28]:
def generate_noise(size, seed):
    if seed == 0:
        randomness = random.randint(0, 1000)
    else:
        randomness = seed
    map = np.zeros(size)

    x_offset = random.uniform(0, 1000)
    y_offset = random.uniform(0, 1000)

    scale = random.uniform(50, 150)
    for i in range(size[0]):
        for j in range(size[1]):
            map[i][j] = noise.pnoise2(
                (i + x_offset) / scale,
                (j + y_offset) / scale,
                octaves=5,
                persistence=0.5,
                lacunarity=2.0,
                base=randomness
            )
    return map

from collections import deque

def has_valid_path(arr, start, end):
    rows, cols = arr.shape
    visited = [[False] * cols for _ in range(rows)]
    queue = deque([start])

    while queue:
        x, y = queue.popleft()

        if (x, y) == end:
            return True

        for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < rows and 0 <= ny < cols:
                if arr[nx][ny] != 0 and not visited[nx][ny]:
                    visited[nx][ny] = True
                    queue.append((nx, ny))

    return False

def generate_map(size, seed):
    map = generate_noise(size, seed)
    map = set_map_numbers(map, 1)
    return map

def generate_valid_map(size, seed):
    map_shape = size

    map_start = (int(map_shape[0] / 2), 0)
    map_end = (int(map_shape[0] / 2), map_shape[1] - 1)

    valid_map_found = False
    map_counter = 1
    while not valid_map_found:
        print("Currently chacking map: " + str(map_counter))
        map = generate_map(map_shape, seed)
        valid_map_found = has_valid_path(map, map_start, map_end)
        map_counter += 1
    print("found a valid map on the: " + str(map_counter - 1) + "th try")
    return map

map = generate_valid_map((100, 200), 0)
print(map)


Currently chacking map: 1
Currently chacking map: 2
Currently chacking map: 3
found a valid map on the: 3th try
[[1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


Ik heb ook een functie gemaakt om de array te kunnen visualizen, dit doe ik doormiddel van PILLOW, een library die gemaakt is voor het maken, manipuleren en opslaan van images.

In [ ]:
print("the input map is: " + str(map))
def show_map(map, save):
    if save:
        if im.mode != 'RGB':
            im = im.convert('RGB')
        im.save("map.jpg")
    processed_map = set_map_numbers(map, 255)
    im = Image.fromarray(processed_map)
    im.show()


show_map(map, False)
print(map)

the input map is: [[255. 255. 255. ...   0.   0.   0.]
 [255. 255. 255. ...   0.   0.   0.]
 [255. 255. 255. ...   0.   0.   0.]
 ...
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]]
[[255. 255. 255. ...   0.   0.   0.]
 [255. 255. 255. ...   0.   0.   0.]
 [255. 255. 255. ...   0.   0.   0.]
 ...
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]]


In [38]:
data = {
    "map_data" : map.tolist()
}

with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)
